In [1]:
using ITensors 
using Plots
using Measurements   
using DelimitedFiles 

In [2]:
# lattice_large - lattice_small = 2 
function GrossNeveu(L::Int64,N::Int64,m::Float64,G²::Float64)
    l = L-2;
    sites = siteinds("Fermion",L)
    ampoₗ = OpSum()
    ampoₛ = OpSum()
    # Large Lattice 
    for j in 1:L-N
        ampoₗ -= 1im,"C",j,"Cdag",j+N
        ampoₗ += 1im,"C",j+N,"Cdag",j
    end
    for j in 1:N:L-N
        ampoₗ += (G²/2),"C * Cdag",j
    end
    for j in 1:L
        ampoₗ += (m*(-1)^j),"C * Cdag",j
    end
    Hₗ = MPO(ampoₗ, sites)
    ψₗ₀ = randomMPS(sites,100)
    ######################################
    ## small lattice 
    for j in 1:l-N
        ampoₛ -= 1im,"C",j,"Cdag",j+N
        ampoₛ += 1im,"C",j+N,"Cdag",j
    end
    for j in 1:N:l-N
        ampoₛ += (G²/2),"C * Cdag",j
    end
    for j in 1:l
        ampoₛ += (m*(-1)^j),"C * Cdag",j 
    end
    Hₛ = MPO(ampoₛ,sites[1:l])
    ψₛ₀ = randomMPS(sites[1:l],100)
#     sweep = Sweeps(sw)
#     setmaxdim!(sweep,repeat([1024],sw)...)
#     setcutoff!(sweep,1E-10)
    sweep = Sweeps([
         "maxdim" "mindim" "cutoff" 
          100      10      1e-16    
          200      10      1e-16    
          500      20      1e-18    
          1000     30      1e-20    
          1024     50      1e-22
         ])
#     sweep = Sweeps([
#          "maxdim" "mindim" "cutoff" 
#           1024      10       1e-16    
#           1024      10      1e-16    
#           1024      20      1e-18    
# #           1000     30      1e-20    
# #           1024     50      1e-22
#          ])
    eₗ₀,ψₗ = dmrg(Hₗ,ψₗ₀,sweep,outputlevel=1)
    eₛ₀,ψₛ = dmrg(Hₛ,ψₛ₀,sweep,outputlevel=1)
    # error computation
    nrm = norm(inner(Hₛ,ψₛ,Hₛ,ψₛ) - inner(ψₛ',Hₛ,ψₛ)^2)
    #  Overlap computation
    v1 = ITensor(sites[l+1]) 
    for j in 1:size(v1)[1]
        v1[j] = 1/sqrt(2)
    end
    v2 = ITensor(sites[L])
    for j in 1:size(v2)[1]
        v2[j] = 1/sqrt(2)
    end
    v = ITensor(1.0)
    for j in 1:l
        v = v*dag(ψₛ[j])*ψₗ[j]
    end
    v = v*v1*ψₗ[l+1]
    v = v*v2*ψₗ[L]
    return inner(v,v), norm(v), nrm 
end

GrossNeveu (generic function with 1 method)

In [63]:
result_inner, result_norm, result_error = nothing, nothing, nothing

(nothing, nothing, nothing)

In [64]:
# L::Integer, N::Integer,G²::Float64,m::Float64,sw::Integer
result_inner = Vector{ComplexF64}();
result_norm = Vector{ComplexF64}();
result_error = Vector{ComplexF64}();

In [65]:
# GrossNeveu(L,N,m,G²)
for j in 4:30
    println(j)
    nnr , nrm, err = GrossNeveu(j,1,0.5,1.0); # change BD --> 100
    push!(result_inner, nnr);
    push!(result_norm, nrm);
    push!(result_error, err);
end 

4
After sweep 1 energy=-1.6289789414104883  maxlinkdim=4 maxerr=0.00E+00 time=0.008
After sweep 2 energy=-1.6293923474715877  maxlinkdim=4 maxerr=0.00E+00 time=0.007
After sweep 3 energy=-1.6293928359493401  maxlinkdim=4 maxerr=0.00E+00 time=0.010
After sweep 4 energy=-1.6293928365446382  maxlinkdim=4 maxerr=0.00E+00 time=0.009
After sweep 5 energy=-1.6293928365453647  maxlinkdim=4 maxerr=0.00E+00 time=0.011
After sweep 1 energy=-0.7804983140825906  maxlinkdim=2 maxerr=0.00E+00 time=0.004
After sweep 2 energy=-0.7807763925525854  maxlinkdim=2 maxerr=0.00E+00 time=0.003
After sweep 3 energy=-0.7807764064037251  maxlinkdim=2 maxerr=0.00E+00 time=0.003
After sweep 4 energy=-0.780776406404415  maxlinkdim=2 maxerr=0.00E+00 time=0.003
After sweep 5 energy=-0.7807764064044144  maxlinkdim=2 maxerr=0.00E+00 time=0.003
5
After sweep 1 energy=-2.3262197571028502  maxlinkdim=4 maxerr=0.00E+00 time=0.016
After sweep 2 energy=-2.33683068379098  maxlinkdim=4 maxerr=0.00E+00 time=0.015
After sweep 3 e

14
After sweep 1 energy=-6.115833866128015  maxlinkdim=100 maxerr=3.53E-07 time=0.769
After sweep 2 energy=-6.118475403822556  maxlinkdim=125 maxerr=6.31E-17 time=0.770
After sweep 3 energy=-6.119100718354324  maxlinkdim=127 maxerr=4.05E-19 time=0.830
After sweep 4 energy=-6.119240255734539  maxlinkdim=128 maxerr=0.00E+00 time=1.075
After sweep 5 energy=-6.119262846359917  maxlinkdim=128 maxerr=0.00E+00 time=0.739
After sweep 1 energy=-5.211429231512668  maxlinkdim=64 maxerr=0.00E+00 time=0.259
After sweep 2 energy=-5.215909564886794  maxlinkdim=64 maxerr=0.00E+00 time=0.232
After sweep 3 energy=-5.216557152477995  maxlinkdim=64 maxerr=0.00E+00 time=0.248
After sweep 4 energy=-5.216632666838036  maxlinkdim=64 maxerr=0.00E+00 time=0.249
After sweep 5 energy=-5.216637051645484  maxlinkdim=64 maxerr=0.00E+00 time=0.251
15
After sweep 1 energy=-6.8457889429368395  maxlinkdim=100 maxerr=8.68E-07 time=1.439
After sweep 2 energy=-6.851248687560317  maxlinkdim=128 maxerr=0.00E+00 time=1.279
Af

After sweep 4 energy=-9.557478516761995  maxlinkdim=1000 maxerr=6.46E-18 time=79.595
After sweep 5 energy=-9.55952160273805  maxlinkdim=1024 maxerr=0.00E+00 time=93.571
24
After sweep 1 energy=-10.60690533193851  maxlinkdim=100 maxerr=4.55E-04 time=5.548
After sweep 2 energy=-10.627489137586595  maxlinkdim=200 maxerr=5.17E-11 time=14.987
After sweep 3 energy=-10.629468678860626  maxlinkdim=500 maxerr=5.18E-14 time=51.765
After sweep 4 energy=-10.630677260221221  maxlinkdim=1000 maxerr=4.15E-15 time=218.209
After sweep 5 energy=-10.631581703894634  maxlinkdim=1024 maxerr=1.05E-14 time=289.711
After sweep 1 energy=-9.695509508590815  maxlinkdim=100 maxerr=2.63E-04 time=4.304
After sweep 2 energy=-9.717055640748024  maxlinkdim=200 maxerr=1.00E-11 time=12.288
After sweep 3 energy=-9.721441030139372  maxlinkdim=500 maxerr=2.35E-14 time=37.776
After sweep 4 energy=-9.72526766786161  maxlinkdim=1000 maxerr=8.48E-16 time=117.676
After sweep 5 energy=-9.727571393846953  maxlinkdim=1024 maxerr=2

In [66]:
result_inner2, result_norm2, result_error2 = nothing, nothing, nothing

(nothing, nothing, nothing)

In [67]:
site_number = [x for x in 1:length(result_inner)];
result_inner2 = real(result_inner);
result_norm2 = real(result_norm);
result_error2 = real(result_error);

In [68]:
error_bar = nothing

In [69]:
error_bar = [sqrt(x) for x in result_error2[1:end]];

In [80]:
scatter(site_number, 
    result_norm2.± error_bar, 
#     xlim=(0,30),xticks=((0:5:30)), 
    ylim=(0.36,0.49), 
    yticks=((0.36:0.01:0.49)),   
    markersize=2,
    xlabel="Site Number",
    ylabel="Inner Product",
    label="m₀=0.5,G²=1.0",
    title="Overlap")
# savefig("m05.svg")

"D:\\Amin\\GN\\G^2\\1\\m05.svg"

In [81]:
# writedlm("m05_inner.txt",result_inner2)
# writedlm("m05_norm.txt",result_norm2)
# writedlm("m05_error.txt",result_error2)